<div><h3>06. Model Deployment</h3></div>

This will make use of __TorchServe__, a model serving (i.e., deployment) package within the PyTorch ecosystem. To deploy a model via __TorchServe__, we need a __Model Handler__ which will tell our the _torchserve_ server how to handle our model. It will contain four methods:

1. __Initialize__: Where all required utilities are initialized.

2. __Preprocess__: Where the data provided by the user is prepared before being passed to the model.

3. __Inference__: Where the model prediction process can be configured.

4. __Postprocess__: Where the model predictions are processed as needed e.g., converting numbers to predicted classes.
  
__Note__: Torchserve provided ready made Handlers for these. We will design our custom Handlers though.


---
Torchserve has two main API ports of interest to us in this video:

1. __Management API__: Port 8081. This is the port via which the server can be configured and its behaviour managed.

2. __Inference API__: Port 8080. This is the port via wich the model returns predictions.

---
#### High-level deployment steps:

1. Register the model.

2. Write the handler.

3. Start the server and point it to the necessary inputs e.g. registered model, model handler, etcetera.

4. Run inference.

In [2]:
%%writefile handler_script.py
import numpy as np
import torch
from ts.torch_handler.image_classifier import ImageClassifier
from torch.profiler import ProfilerActivity

from torchvision import transforms as T
from imageio.v3 import imread

class CassavaClassifier(ImageClassifier):
    transform = T.Compose(
        [
            T.Resize((224, 224)),
            T.Normalize(mean = [.485, .456, .406], std = [.229, .224, .225])
        ]
    )

    class_map = {
        "cbsd": 0,
        "cbb": 1,
        "cmd": 2,
        "cgm": 3,
        "healthy": 4
    }
    class_map = {v: k for k, v in class_map.items()}

    def __init__(self, ):
        super().__init__()

    @torch.no_grad()
    def preprocess(self, data):
        data = data[0]
        data = bytes(data["body"])

        data = imread(data)
        data = torch.tensor(data, dtype = torch.float32)

        data = data.permute(2, 0, 1)
        data = self.transform(data).unsqueeze(0)

        return data.cuda()

    @torch.no_grad()
    def postprocess(self, data):
        class_indices = data.max(dim = -1).indices.cpu().numpy().tolist()
        class_names = [f"\nPredicted Class: {self.class_map[class_index]}\n" for class_index in class_indices]
        return class_names

Overwriting handler_script.py
